In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import keras as k

Using TensorFlow backend.


In [3]:
dimer_atoms = 4
solvent_atoms = 72
numberOfLayers = 1
total_atoms = dimer_atoms + solvent_atoms
neurons = 3
dim = 2

In [ ]:
x = tf.placeholder(tf.float32, [None, dimer_atoms + solvent_atoms - 1], name = 'input' )
y = tf.placeholder(tf.float32, [None, 2* solvent_atoms], name='output')
w = tf.Variable(tf.random.truncated_normal([total_atoms - 1],mean=0, stddev= 2 ), name='weights1' ) 
'''
for i in range(solvent_atoms):
    w.append ( tf.Variable(tf.random.truncated_normal([total_atoms - 1],mean=0, stddev=1 / np.sqrt(n_atoms)), name='weights1' ) )
w= tf.Variable(w)
'''
hBeforeActivation = tf.multiply( w , x[:-1] ) #element wise multiplication
h = tf.nn.relu( tf.cumsum(hBeforeActivation) ) #cumulativeSum


In [ ]:
v = tf.Variable(tf.random.truncated_normal([total_atoms -1, 2* solvent_atoms ], mean=0, stddev=2))
y = tf.linalg.matvec(tf.transpose(v), h )

In [ ]:
def loss(yPred, output):
    normal = tfp.trainable_distributions.normal(output[0])
    loss = -tf.reduce_mean(normal.log_prob(xInput))
    train_op = tf.train.AdamOptimizer(learning_rate=2.**-5).minimize(loss)
    mse = tf.reduce_mean(tf.squared_difference(y, normal.mean()))


In [ ]:
batch_size = 128
epochs = 100

In [ ]:
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',
              metrics=['myloss'])

In [4]:
import torch
import torch.nn as nn
from pathlib import Path
import torchvision.transforms as transforms
import torch.optim as optim


In [103]:
#RNADE
class RNADE(nn.Module):
    def __init__(self,dimer_atoms,solvent_atoms):
        super(RNADE, self).__init__()
        self.dimer_atoms = dimer_atoms
        self.solvent_atoms = solvent_atoms
        self.total_dims = self.dimer_atoms + self.solvent_atoms
        self.i2h = nn.Linear(self.total_dims-1, self.total_dims-1)
        self.h2o = nn.Linear(self.total_dims-1, self.total_dims-self.dimer_atoms)
        self.D = self.total_dims
        self.H = 128
        self.params = nn.ParameterDict({
            "V" : nn.Parameter(torch.randn(self.D, self.H)),
            "b" : nn.Parameter(torch.zeros(self.D)),
            "V2" : nn.Parameter(torch.randn(self.D, self.H)),
            "b2" : nn.Parameter(torch.zeros(self.D)),
            "W" : nn.Parameter(torch.randn(self.H, self.D)),
            "c" : nn.Parameter(torch.zeros(1, self.H)),
        })
        nn.init.xavier_normal_(self.params["V"])
        nn.init.xavier_normal_(self.params["V2"])
        nn.init.xavier_normal_(self.params["W"])
        
    def forward(self, x):
        ai = self.params["c"]   
        print(ai.size())
        final_prob = []
        for d in range(self.D):
            h_i = torch.relu(ai)
            print(h_i.size())
            alphai = torch.exp( self.params["V"].mm(h_i[d:d+1,:].t() ) + self.params["b"][d] )
            meani = self.params["V2"].mm(h_i[d,:].t() ) + self.params["b2"][d]
            print(meani.size())
            final_prob.append([alphai, meani])
            ai = x[:, d].mm(self.params["W"][:, d].t()) + ai
        #print(final_prob)
        print(len(final_prob))
        return final_prob

In [104]:
from math import *

In [105]:
def lossFunct(output,pred):
    alpha = output[:,1:,1]
    mean = output[:,1:,0]
    loss = torch.exp(0.5 *  ( (pred[:,1:]- mean)/alpha)**2  )/ (sqrt(2 *3.14) * alpha)
    return -torch.log( loss ).sum(axis= 1).sum()

In [106]:
print(device)

cpu


In [107]:
model = RNADE(4,72).to(device)
loss = lossFunct

In [108]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [109]:
model.train()

RNADE(
  (i2h): Linear(in_features=75, out_features=75, bias=True)
  (h2o): Linear(in_features=75, out_features=72, bias=True)
  (params): ParameterDict(
      (V): Parameter containing: [torch.FloatTensor of size 76x128]
      (V2): Parameter containing: [torch.FloatTensor of size 76x128]
      (W): Parameter containing: [torch.FloatTensor of size 128x76]
      (b): Parameter containing: [torch.FloatTensor of size 76]
      (b2): Parameter containing: [torch.FloatTensor of size 76]
      (c): Parameter containing: [torch.FloatTensor of size 1x128]
  )
)

In [110]:

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print(data.size())
        optimizer.zero_grad()
        x_hat = model(data)
        loss = lossFunct(x_hat, data)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

torch.Size([256, 76])
torch.Size([256, 128])
torch.Size([256, 128])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
a = torch.tensor([[ [0, 2], [0,1], [0,0.5] ], [ [0, 2], [0,1], [0,0.5] ] ] )

In [ ]:
output = torch.tensor( [ [1, 2,3], [3,1, 2]] )

In [ ]:
lossFunct(a,output)

In [ ]:
torch.tensor([[1,2], [2,3],[3,4]]).t()

In [8]:
trajdict = np.load('output.npz')
import ast
#params = ast.literal_eval(str(trajdict['params']))
traj_closed_train = trajdict['traj_closed_train_hungarian']
traj_open_train = trajdict['traj_open_train_hungarian']
traj_closed_test = trajdict['traj_closed_test_hungarian']
traj_open_test = trajdict['traj_open_test_hungarian']
x = np.vstack([traj_closed_train, traj_open_train])
xval = np.vstack([traj_closed_test, traj_open_test])

In [9]:
batch_size =256
epochs = 10
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

In [10]:
train_set = traj_closed_train

In [11]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
data_path = Path(".").absolute().parent / "data"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
device = "cpu"